In [8]:
!pip install simplegmail
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.6 MB/s 
     |████████████████████████████████| 182 kB 46.5 MB/s 
     |████████████████████████████████| 7.6 MB 20.2 MB/s 


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/data/Movie_Dialogpt/medium/models/training_args.bin')
if torch.cuda.is_available():
    device = torch.device('cuda:0') # GPU
else:
    device = torch.device('cpu') # CPU
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout)

In [32]:
def reply_message(message_list):
  base_tokens = []
  for message in message_list:
    tmp_tokens = tokenizer.encode(message) + [tokenizer.eos_token_id]
    base_tokens = base_tokens + tmp_tokens
  predicted_tokens = model.generate(torch.tensor([base_tokens], device=device), max_length=200, pad_token_id=tokenizer.eos_token_id)
  reply_tokens = predicted_tokens[0][len(base_tokens):]
  return tokenizer.decode(reply_tokens, skip_special_tokens=True)

In [34]:
import logging
from time import sleep, time

from simplegmail import Gmail


class SessionData:
    EXPIRE_TIME = 120  # 30 minutes

    def __init__(self):
        self.data = {}

    def save_received_and_get(self, number, message):
        now = time()
        session_with_time = self.data.get(number)
        if session_with_time is None:
            session = []
        else:
            (session, update_time) = session_with_time
            if now - update_time > self.EXPIRE_TIME:
                session = []
        session.append((0, message))
        self.data[number] = (session, now)
        return session

    def save_sent(self, number, message):
        (session, update_time) = self.data.get(number)
        session.append((1, message))
        self.data[number] = (session, time())


def main():
    logging.basicConfig(level=logging.DEBUG)
    gmail = Gmail()
    session_data = SessionData()

    while(1):

        logging.info('fetching sms')
        unread_messages = gmail.get_unread_inbox()
        logging.debug(f'{len(unread_messages)} mails found')

        for message in unread_messages:
            if not message.sender.endswith('@txt.voice.google.com>'):
                logging.debug('sender does not match, pass')
                message.mark_as_read()
                continue
            sender_mail = message.sender.split('<')[1][:-1]
            sender_number = sender_mail[13:23]
            text = message.plain.split('<https://voice.google.com>', 1)[1].split(
                'YOUR ACCOUNT <https://voice.google.com> HELP CENTER', 1)[0].strip()
            if text.endswith(
                    'To respond to this text message, reply to this email or visit Google Voice.'):
                text = text[:-75]
            logging.info('sms from '+sender_number+': '+text)
            session = session_data.save_received_and_get(sender_number, text)
            reply = handle_message(session)
            logging.info('replying: '+reply)
            session_data.save_sent(sender_number, reply)
            gmail.send_message(
                to=sender_mail,
                sender="fujiwaranominglei@gmail.com",
                subject='reply',
                msg_plain=reply,
                msg_html=reply,
                signature=True
            )
            logging.info('completed')
            message.mark_as_read()
        sleep(5)


def handle_message(session):
    # format of session:
    # session is a list, each of the item is
    # a tuple, each tuple represent a sent or
    # received message. the tuple contain two
    # item, first one is the flag of sent (1)
    # or receive (0), second item is the message
    # string
    #
    # example:
    # session = [(0, 'Hello, how are you'), (1, 'Never better'), (0, 'That's cool')]
    message_list = [message[1] for message in session]
    reply = reply_message(message_list)
    print(reply)
    if len(reply) == 0:
      reply = '......'
    return reply


if __name__ == '__main__':
    main()


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


I'm not going to hurt you.



A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


I didn't know you were so sensitive.



A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


I don't know.



A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


I don't know.



KeyboardInterrupt: ignored

In [3]:
!python -V

Python 3.7.15
